# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [16]:
"""
    Creates table for question 1: 
    (Give me the artist, song title and song's length 
    in the music app history that was heard during 
    sessionId = 338, and itemInSession = 4)
    
    Primary Key:
    - session_id - unique session identifier as integer
    - item_in_session - unique session item identifier as integer
    using just the session_id is not unique enough. There is more than one song per session. 
    The query would not return all the data.
    by adding item_in_session as a clustering column to make the composite key is unique enough to 
    that the query will return all the data.
    """
query = "CREATE TABLE IF NOT EXISTS artist_song_session_id "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)


                    

In [17]:
""" Extract the data from event_datafile_new.csv using
csv reader library package

"""
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
""" insert data into artist_song_session_id table using insert query
convert strings to int and floats at session.execute
"""
        
        query = "INSERT INTO artist_song_session_id (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [18]:
"""isolate artist_name, song_title, song_length from artist_song_session_id table by session id and item_in_session
"""
query = "SELECT artist_name, song_title, song_length FROM artist_song_session_id WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [21]:
"""
    Creates table for question 2: 
    (Give me only the following: name of artist, 
    song (sorted by itemInSession) and user (first and last name)
    for userid = 10, sessionid = 182)
    
    Primary Key:
    - user_id - unique user identifier as integer
    - session_id - unique session identifier as integer
    - item_in_session - unique session item identifier as integer
    using just the user_id is not unique enough. There is more than one session per user and more than one song per session. 
    The query would not return all the data.
    by using user_id and session_id as a composite key and item_in_session as a clustering column is unique enough to 
    that the query will return all the data.
    """
query = "CREATE TABLE IF NOT EXISTS artist_song_session_per_user"
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, user_first_name text, user_last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

""" Extract the data from event_datafile_new.csv using
csv reader library package

"""
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
""" insert data into artist_song_session_per_user table using insert query
convert strings to int and floats at session.execute
""" 

        query = "INSERT INTO artist_song_session_per_user (user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))     
        
"""isolate artist_name, song_title, user_first_name, user_last_name from artist_song_session_per_user table by user id and session id and sorted by item_in_session
"""
query = "SELECT artist_name, song_title, user_first_name, user_last_name FROM artist_song_session_per_user WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.user_first_name, row.user_last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [30]:
"""
    Creates table for question 3: 
    (Give me every user name (first and last) in my music 
    app history who listened to the song 'All Hands Against His Own')
    
    Primary Key:
    - song_tile - tile of each song as text
    - user_id - unique user identifier as integer

    we need to sort the data by user_id. 
   by using song_title and user_id as a composite key with user_id as a clustering column is unique enough to 
    that the query will return all the data in the order we want it.
    """
query = "CREATE TABLE IF NOT EXISTS user_per_song "
query = query + "(song_title text, user_id int, session_id int, item_in_session int, user_first_name text, user_last_name text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

""" Extract the data from event_datafile_new.csv using
csv reader library package

"""
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
"""isolate song_title for each user_first_name, user_last_name from user_per_song table
"""
        query = "INSERT INTO user_per_song (song_title, user_id, session_id, item_in_session,  user_first_name, user_last_name)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), int(line[8]), int(line[3]), line[1], line[4]))  
        
query = "SELECT user_id, user_first_name, user_last_name FROM user_per_song WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name, row.user_last_name)        
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


Down To The Bone Keep On Keepin' On Sylvie Cruz 10 182 0
Three Drives Greece 2000 Sylvie Cruz 10 182 1
Sebastien Tellier Kilometer Sylvie Cruz 10 182 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 10 182 3


### Drop the tables before closing out the sessions

In [1]:
query = "drop table artist_song_session_id"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

name 'session' is not defined


In [27]:
query = "drop table artist_song_session_per_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e) 

In [29]:
query = "drop table user_per_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()